In [1]:
# ingestion_app.py

# Import of required modules
from config import *
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.text_embeddings_inference import TextEmbeddingsInference
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.node_parser import SimpleNodeParser, MarkdownNodeParser
from llama_index.vector_stores.weaviate import WeaviateVectorStore
import weaviate
from tqdm import tqdm
import os

/home/brianda/projects/chatbots/chatbot_assistant_for_citizen_assemblies/orchestator-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
INPUT_DIR = "expert_docs/context_docs_Assembly_1/oficial_docs/transformations01"
# Check if the directory exists
if os.path.exists(INPUT_DIR):
    print(f"El directorio {INPUT_DIR} está listo para ser utilizado.")
else:
    print(f"Error: El directorio {INPUT_DIR} no existe. Verifica la configuración.")

## Initial configuration of the LLM model using config.py variables
#llm_model = Ollama(
#    model=LLM_MODEL_NAME, 
#    keep_alive=LLM_KEEP_ALIVE, 
#    request_timeout=LLM_REQUEST_TIMEOUT, 
#    temperature=LLM_TEMPERATURE, 
#    base_url=LLM_BASE_URL
#)

# Settings for inference of text embeds
embed_model = TextEmbeddingsInference(
    model_name=EMBED_MODEL_NAME,
    base_url=EMBED_BASE_URL,
    timeout=EMBED_TIMEOUT, 
    embed_batch_size=EMBED_BATCH_SIZE
)

# Global configuration for LLM and embedding model
#Settings.llm = llm_model
Settings.embed_model = embed_model

WEAVIATE_URL=WEAVIATE_URL
# Weaviate client configured with URL from config.py
# weaviate_client = weaviate.Client(url=WEAVIATE_URL)
weaviate_client = weaviate.connect_to_local()

El directorio expert_docs/context_docs_Assembly_1/oficial_docs/transformations01 está listo para ser utilizado.


In [ ]:
print(os.getenv("WEAVIATE_URL"))

: 

In [4]:
# Reading directories and uploading documents
reader = SimpleDirectoryReader(input_dir=INPUT_DIR, recursive=True)
documents = reader.load_data()

# Node parser configured with default values
#parser = SimpleNodeParser.from_defaults(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
parser = MarkdownNodeParser()
nodes = parser.get_nodes_from_documents(documents)

print(f"Proceso de parseo completado. Documentos procesados: {len(documents)}. Nodos generados: {len(nodes)}.")


Proceso de parseo completado. Documentos procesados: 129. Nodos generados: 129.


In [5]:
# Configuration of the vector store and storage context
if weaviate_client.collections.exists(INDEX_NAME):
    print(f"Se eliminará el índice anterior: {INDEX_NAME}")
    weaviate_client.collections.delete(INDEX_NAME)

Se eliminará el índice anterior: Documents_md2


In [7]:
EMBEDDING_DIM

768

In [8]:
print(f"Configurando y construyendo el índice: {INDEX_NAME}")
vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                   index_name=INDEX_NAME, 
                                   embed_model=embed_model,
                                   embedding_dim=EMBEDDING_DIM,
                                   text_key="content")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


Configurando y construyendo el índice: Documents_md2


In [9]:

# Index configuration and construction with parsed data
index = VectorStoreIndex(nodes, storage_context=storage_context, show_progress=True)

print(f"Índice {INDEX_NAME} construido con éxito.")


Generating embeddings: 100%|██████████| 129/129 [00:07<00:00, 16.37it/s]


Índice Documents_md2 construido con éxito.


In [10]:
jeopardy = weaviate_client.collections.get(INDEX_NAME)
response = jeopardy.query.fetch_objects()

for o in response.objects:
    print(o.properties)

{'file_size': 91194.0, 'content': 'Posicionamento 1: Modelo descentralizado\n\nModelo energético totalmente descentralizado con instalaciones de energías renovables a pequeña escala\n\n**Carles Riba**\n\nIngeniero industrial, profesor emérito de la Universidad Politécnica de Cataluña (UPC) y presidente del Colectivo por un Nuevo Modelo Energético Social y Sostenible (CMES).', '_node_type': 'TextNode', 'last_modified_date': '2024-05-12', 'document_id': UUID('80004219-6ee7-471c-ab36-1a7eebbc3cd2'), 'relationships': None, 'ref_doc_id': '80004219-6ee7-471c-ab36-1a7eebbc3cd2', 'doc_id': UUID('80004219-6ee7-471c-ab36-1a7eebbc3cd2'), 'text': None, 'file_type': 'text/markdown', 'file_path': '/home/brianda/projects/chatbots/chatbot_assistant_for_citizen_assemblies/services/ingestion/expert_docs/context_docs_Assembly_1/oficial_docs/transformations01/accc_kit-informativo_energia_v2_esp.md', 'creation_date': '2024-05-12', 'file_name': 'accc_kit-informativo_energia_v2_esp.md', '_node_content': '{"i

In [11]:
print(jeopardy.aggregate.over_all())


AggregateReturn(properties={}, total_count=129)


In [12]:
# ------------------------------
# Direct search test with Weaviate
print("Consulta de prueba: ¿Cuáles son las desventajas de la ganadería intensiva?")

question = "Desventajas de la ganadería intensiva"

documents = weaviate_client.collections.get(INDEX_NAME)



Consulta de prueba: ¿Cuáles son las desventajas de la ganadería intensiva?


In [13]:
print(f"EMBEDDING_DIM: {os.getenv('EMBEDDING_DIM')}")


EMBEDDING_DIM: 768


In [14]:
#response = documents.query.near_text(question).get()

response = documents.query.near_text(query=question,
                                     include_vector=True,
                                     limit=4)


#print(response.objects[0].properties)  # Inspect the first object


WeaviateQueryError: Query call with protocol GRPC search failed with message explorer: get class: vector search: object vector search at index documents_md2: shard documents_md2_4wBsflVohoSR: vector search: knn search: distance between entrypoint and query node: vector lengths don't match: 768 vs 384.

In [14]:
# ------------------------------
# Direct search test with Weaviate
print("Consulta de prueba: ¿Cuáles son las desventajas de la ganadería intensiva?")

results = weaviate_client.query.get(INDEX_NAME, ["content", "_additional { score }"]).with_bm25(
    query="¿Cuáles son las desventajas de la ganadería intensiva?"
).with_limit(5).do()

# Imprimir los resultados obtenidos
print("Resultados de la consulta:")
for i, result in enumerate(results['data']['Get'][INDEX_NAME], start=1):
    content = result['content']
    score = result['_additional']['score']
    print(f"{i}. Content: {content} - Score: {score}")
# Question test with LlamaIndex
weaviate_client = weaviate.Client(url=WEAVIATE_URL)
vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                   index_name=index_name,
                                   text_key="content")


Consulta de prueba: ¿Cuáles son las desventajas de la ganadería intensiva?


AttributeError: 'WeaviateClient' object has no attribute 'query'

In [ ]:
index = VectorStoreIndex.from_vector_store(vector_store)
query_engine = index.as_query_engine(similarity_top_k=5)

result = query_engine.query("¿Cuáles son las ventajas y desventajas un modelo de energía de grandes instalaciones? Responde en español. Utiliza un formato de tabla markdown. Si hay opiniones de diferentes personas indica su nombre.")
print(result.response)

weaviate_client.close()

In [ ]:
# ------------------------------

# Direct search test with Weaviate
# No se puede realizar la consulta debido a que el índice no se ha creado con la imagen de Weaviate del modelo (porque no funciona con llama_index)
#print("Consulta de prueba: ¿Cuáles son las desventajas de la ganadería intensiva?")
#
#results = weaviate_client.query.get(INDEX_NAME, ["content", "_additional { score }"]).with_bm25(
#    query="¿Cuáles son las desventajas de la ganadería intensiva?"
#).with_limit(5).do()
#
## Imprimir los resultados obtenidos
#print("Resultados de la consulta:")
#for i, result in enumerate(results['data']['Get'][INDEX_NAME], start=1):
#    content = result['content']
#    score = result['_additional']['score']
#    print(f"{i}. Content: {content} - Score: {score}")

# Question test with LlamaIndex
#weaviate_client = weaviate.Client(url=WEAVIATE_URL)
#vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
#                                   index_name=index_name,
#                                   text_key="content")